In [ ]:
!pip install spacy transformers nltk
!python -m spacy download en_core_web_sm
import nltk
nltk.download('punkt')


In [ ]:
import spacy
from heapq import nlargest
from transformers import pipeline
import re
from nltk.tokenize import sent_tokenize


In [ ]:
def preprocess_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
def extractive_summary(text, num_sentences=3):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentence_scores = {}

    for sent in doc.sents:
        for word in sent:
            if word.is_stop == False and word.is_alpha:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + 1

    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    final_summary = ' '.join([str(sent) for sent in summary_sentences])
    return final_summary


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summary(text):
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']


In [ ]:
article = """
The World Health Organization (WHO) declared a global health emergency following the outbreak of a novel respiratory virus that has rapidly spread across continents.
The virus, first identified in a small city, has now been confirmed in over 40 countries, with thousands of reported infections and increasing fatalities.
International travel restrictions have been implemented, and governments are urging citizens to take precautions such as wearing masks and avoiding large gatherings.
Scientists are working around the clock to develop a vaccine, while healthcare systems are preparing for potential surges in patients.
WHO officials emphasized the importance of global cooperation and transparent information sharing to combat the outbreak effectively.
"""


In [ ]:
cleaned_text = preprocess_text(article)

# Extractive
print("🔹 Extractive Summary:\n")
print(extractive_summary(cleaned_text))

# Abstractive
print("\n🔹 Abstractive Summary:\n")
print(abstractive_summary(cleaned_text))
